In [18]:
import pygame
import pyaudio
import wave
import time
pygame.init()
import numpy as np
import pygame, mutagen.mp3


CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "data/output.wav"

p = pyaudio.PyAudio()

four4 = pygame.mixer.Sound("data/startBeat.wav")

In [19]:
rapList = ["data/rapback1.wav","data/rapback2.wav","data/rapback3.wav","data/rapback4.wav","data/rapback5.wav",
          "data/rapback6.wav","data/rapback7.wav","data/rapback8.wav","data/rapback9.wav","data/rapback10.wav",
          "data/rapback11.wav","data/rapback12.wav","data/rapback13.wav","data/rapback14.wav","data/rapback15.wav",
          "data/rapback16.wav","data/rapback17.wav","data/rapback18.wav","data/rapback19.wav","data/rapback20.wav",
          "data/rapback21.wav","data/rapback22.wav","data/rapback23.wav","data/rapback24.wav"]
rapsongs = len(rapList)
s1rap = np.random.randint(1,rapsongs,1)
rapSong= pygame.mixer.Sound(rapList[s1rap[0]]) 

In [20]:
popList = ["data/output000.mp3","data/output001.mp3","data/output003.mp3","data/output004.mp3",
          "data/output005.mp3","data/output006.mp3","data/output007.mp3","data/output008.mp3","data/output009.mp3",
          "data/output011.mp3","data/output012.mp3","data/output013.mp3","data/output014.mp3",
          "data/output015.mp3","data/output016.mp3","data/output017.mp3","data/output018.mp3","data/output019.mp3",
          "data/output020.mp3","data/output021.mp3","data/output022.mp3","data/output023.mp3","data/output024.mp3",
          "data/output025.mp3","data/output026.mp3","data/output027.mp3","data/output028.mp3","data/output029.mp3",
          "data/output030.mp3","data/output031.mp3","data/output032.mp3","data/output033.mp3","data/output034.mp3",
          "data/output035.mp3","data/output036.mp3","data/output037.mp3","data/output038.mp3","data/output039.mp3"]
popsongs = len(popList)
s1 = np.random.randint(1,popsongs,1)

mp3 = mutagen.mp3.MP3(popList[s1[0]])
pygame.mixer.init(frequency=mp3.info.sample_rate)

pygame.mixer.music.load(popList[s1[0]])
popSong =pygame.mixer.music


In [21]:
from keras import backend as K
import os
import time
import h5py
import sys
from tagger_net import MusicTaggerCRNN
from keras.optimizers import SGD
import numpy as np
from keras.utils import np_utils
from math import floor
from music_tagger_cnn import MusicTaggerCNN
from utils import save_data, load_dataset, save_dataset, sort_result, predict_label, extract_melgrams
import matplotlib.pyplot as plt
K.set_image_dim_ordering('th')

In [22]:
TEST = 1

LOAD_MODEL = 0
LOAD_WEIGHTS = 1
MULTIFRAMES = 1
time_elapsed = 0

# GTZAN Dataset Tags
#tags = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
tags = ['pop', 'pop', 'pop', 'pop', 'hiphop', 'hiphop', 'hiphop', 'pop', 'pop', 'pop']
tags = np.array(tags)

# Paths to set
model_name = "example_model"
model_path = "models_trained/" + model_name + "/"
weights_path = "models_trained/" + model_name + "/weights/"

test_songs_list = 'list_example.txt'


In [23]:
def classify():
    # Initialize model
    model = MusicTaggerCRNN(weights=None, input_tensor=(1, 96, 1366))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    if LOAD_WEIGHTS:
        model.load_weights(weights_path+'crnn_net_gru_adam_ours_epoch_40.h5')
        
    X_test, num_frames_test= extract_melgrams(test_songs_list, MULTIFRAMES, process_all_song=False, num_songs_genre='')
        
    num_frames_test = np.array(num_frames_test)
    t0 = time.time()
        
    results = np.zeros((X_test.shape[0], tags.shape[0]))
    predicted_labels_mean = np.zeros((num_frames_test.shape[0], 1))
    predicted_labels_frames = np.zeros((X_test.shape[0], 1))
        
    song_paths = open(test_songs_list, 'r').read().splitlines()
    previous_numFrames = 0
    n=0
        
    for i in range(0, num_frames_test.shape[0]):
        num_frames=num_frames_test[i]
        results[previous_numFrames:previous_numFrames+num_frames] = model.predict(X_test[previous_numFrames:previous_numFrames+num_frames, :, :, :])
            
        s_counter = 0
            
        for j in range(previous_numFrames, previous_numFrames+num_frames):
            total = results[j,:].sum()
            results[j,:]=results[j,:]/total
            sort_result(tags, results[j,:].tolist())
            predicted_label_frames=predict_label(results[j,:])
            predicted_labels_frames[n]=predicted_label_frames
            s_counter += 1
            n+=1
                
        results_song = results[previous_numFrames:previous_numFrames+num_frames]
        mean=results_song.mean(0)
        sort_result(tags, mean.tolist())
        predicted_label_mean=predict_label(mean)
        predicted_labels_mean[i]=predicted_label_mean
            
    return(tags[predicted_label_mean])

    #previous_numFrames = previous_numFrames+num_frames
   

In [24]:
def record():
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    

    stream.stop_stream()

    stream.close()

    p.terminate()


    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
        
    return()


In [ ]:
i=10000
res='null'
four4.play()

while i>0:    
    i=i-1    
    print res   
    
    if(res=='null'):
        print "recording"
        record()
        res = classify()
        continue 
           
    elif(res=='pop'):
        s1 = np.random.randint(1,popsongs,1)
        mp3 = mutagen.mp3.MP3(popList[s1[0]])
        pygame.mixer.init(frequency=mp3.info.sample_rate)

        pygame.mixer.music.load(popList[s1[0]])

        popSong.fadeout(3000)
        rapSong.fadeout(3000)
        four4.fadeout(3000)
        
        popSong =pygame.mixer.music
        popSong.set_volume(10)
        popSong.play()
        
        s2 = np.random.randint(1,rapsongs,1)
        rapSong= pygame.mixer.Sound(rapList[s2[0]])  
        rapSong.set_volume(0.5)
        rapSong.play(fade_ms=1000) 
        
        #print "inside pop"
        res='wait'
        continue
        
        

            
    elif(res=='hiphop'):
        s1 = np.random.randint(1,popsongs,1)
        mp3 = mutagen.mp3.MP3(popList[s1[0]])
        pygame.mixer.init(frequency=mp3.info.sample_rate)

        pygame.mixer.music.load(popList[s1[0]])
        
        popSong.fadeout(3000)
        rapSong.fadeout(3000)
        four4.fadeout(3000)
        popSong =pygame.mixer.music

        popSong.set_volume(10)
        popSong.play()
        
        s2 = np.random.randint(1,rapsongs,1)
        rapSong= pygame.mixer.Sound(rapList[s2[0]])  
        rapSong.set_volume(0.5)
        rapSong.play(fade_ms=1000)
        
        
        #print "inside hiphop"
        
        res='wait'
        continue
        
    elif(res=='wait'):
        print res
        print "inside waiting"
        time.sleep(4)
        res='null'
        continue


null
recording
data/output.wav
num frames: 1
pop : 91.010       pop : 2.007       pop : 1.768       pop : 1.240       pop : 1.062       hiphop : 0.889       hiphop : 0.802       pop : 0.520       hiphop : 0.415       pop : 0.285      
pop : 91.010       pop : 2.007       pop : 1.768       pop : 1.240       pop : 1.062       hiphop : 0.889       hiphop : 0.802       pop : 0.520       hiphop : 0.415       pop : 0.285      
pop
wait
wait
inside waiting
null
recording
data/output.wav
num frames: 1
pop : 91.010       pop : 2.007       pop : 1.768       pop : 1.240       pop : 1.062       hiphop : 0.889       hiphop : 0.802       pop : 0.520       hiphop : 0.415       pop : 0.285      
pop : 91.010       pop : 2.007       pop : 1.768       pop : 1.240       pop : 1.062       hiphop : 0.889       hiphop : 0.802       pop : 0.520       hiphop : 0.415       pop : 0.285      
pop
wait
wait
inside waiting
null
recording
data/output.wav
num frames: 1
pop : 91.010       pop : 2.007       pop : 1.76